# 🚀 한국어 텍스트 교정 - 빠른 데모 (Google Colab)

기존 모델을 활용한 한국어 텍스트 교정 데모
- 빠른 설정 및 테스트
- 사전 훈련된 mT5 모델 활용
- 실시간 텍스트 교정 체험

## ⚡ 빠른 설정

In [ ]:
# GPU 확인 및 기본 설정
!nvidia-smi

# 필요한 패키지 설치
!pip install -q transformers torch gradio sentencepiece

import torch
import time
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import gradio as gr

# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🚀 사용 디바이스: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 🤖 모델 로드

In [ ]:
# 사전 훈련된 mT5 모델 로드 (데모용)
print("📦 mT5 모델 로딩 중...")

model_name = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto" if torch.cuda.is_available() else None
)

if not torch.cuda.is_available():
    model = model.to(device)

model.eval()
print("✅ 모델 로드 완료!")
print("⚠️  참고: 이는 기본 mT5 모델로, 한국어 교정에 특화되지 않았습니다.")
print("   실제 성능을 위해서는 colab_training.ipynb로 파인튜닝을 진행하세요.")

## 🔧 데모 함수

In [ ]:
def simple_correct_text(text: str) -> str:
    """간단한 텍스트 교정 함수 (데모용)"""
    if not text.strip():
        return text
    
    # 한국어 교정을 위한 프롬프트 (기본 mT5 모델용)
    input_text = f"다음 한국어 텍스트의 맞춤법과 문법을 교정하세요: {text.strip()}"
    
    # 토크나이제이션
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        max_length=256,
        truncation=True,
        padding=True
    ).to(device)
    
    # 텍스트 생성
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=256,
            num_beams=3,
            temperature=0.8,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            no_repeat_ngram_size=2
        )
    
    # 결과 디코딩
    corrected = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return corrected.strip()

def demo_correction_with_simple_rules(text: str) -> str:
    """간단한 규칙 기반 교정과 모델 결합 (데모용)"""
    # 기본적인 규칙 기반 교정
    simple_corrections = {
        "안녕하셰요": "안녕하세요",
        "감사햐니다": "감사합니다",
        "괜챠습니까": "괜찮습니까",
        "어떻게 생간하세요": "어떻게 생각하세요",
        "날시가": "날씨가",
        "뫃겠습니다": "부탁드립니다",
        "조을까요": "좋을까요",
        "시작해나요": "시작합니까"
    }
    
    # 규칙 기반 교정 먼저 적용
    corrected_text = text
    for wrong, correct in simple_corrections.items():
        corrected_text = corrected_text.replace(wrong, correct)
    
    # 변경이 있었다면 규칙 기반 결과 반환
    if corrected_text != text:
        return corrected_text
    
    # 변경이 없었다면 모델 사용
    return simple_correct_text(text)

print("✅ 교정 함수 정의 완료")

## 🧪 빠른 테스트

In [ ]:
# 테스트 케이스
test_cases = [
    "안녕하셰요",
    "감사햐니다", 
    "괜챠습니까",
    "어떻게 생간하세요",
    "오늘 날시가 좋네요",
    "점심 메뉴는 뭐가 조을까요"
]

print("🧪 데모 텍스트 교정 테스트")
print("=" * 60)

for i, test_text in enumerate(test_cases, 1):
    print(f"\n{i}. 원본: {test_text}")
    
    # 규칙 기반 + 모델 교정
    start_time = time.time()
    corrected = demo_correction_with_simple_rules(test_text)
    end_time = time.time()
    
    print(f"   교정: {corrected}")
    print(f"   시간: {(end_time - start_time)*1000:.1f}ms")
    print("-" * 50)

## 🖥️ 대화형 데모 인터페이스

In [ ]:
def gradio_demo_correction(text, use_model):
    """Gradio 인터페이스용 교정 함수"""
    if not text.strip():
        return "텍스트를 입력해주세요.", ""
    
    try:
        start_time = time.time()
        
        if use_model:
            # 모델 사용
            corrected = simple_correct_text(text)
            method = "mT5 모델"
        else:
            # 규칙 기반 + 모델 조합
            corrected = demo_correction_with_simple_rules(text)
            method = "규칙 기반 + 모델"
        
        end_time = time.time()
        info = f"방법: {method} | 처리 시간: {(end_time - start_time)*1000:.1f}ms"
        
        return corrected, info
        
    except Exception as e:
        return f"오류 발생: {str(e)}", ""

# Gradio 인터페이스
with gr.Blocks(title="한국어 텍스트 교정 데모") as demo:
    gr.Markdown("# 🇰🇷 한국어 텍스트 교정 데모")
    gr.Markdown(
        "기본 mT5 모델과 간단한 규칙을 활용한 한국어 텍스트 교정 데모입니다.\n"
        "**참고**: 더 나은 성능을 위해서는 한국어 데이터로 파인튜닝된 모델을 사용하세요."
    )
    
    with gr.Row():
        with gr.Column():
            input_text = gr.Textbox(
                label="교정할 텍스트",
                placeholder="교정하고 싶은 한국어 텍스트를 입력하세요...",
                lines=3
            )
            
            use_model = gr.Checkbox(
                value=False,
                label="mT5 모델만 사용 (체크 해제 시 규칙 기반 + 모델 조합 사용)"
            )
            
            correct_btn = gr.Button("교정하기", variant="primary")
            
        with gr.Column():
            output_text = gr.Textbox(
                label="교정된 텍스트",
                lines=3
            )
            processing_info = gr.Textbox(
                label="처리 정보",
                lines=1
            )
    
    # 예시
    gr.Markdown("### 📝 예시 텍스트 (클릭해서 테스트)")
    examples = gr.Examples(
        examples=[
            ["안녕하셰요"],
            ["감사햐니다"],
            ["괜챠습니까"],
            ["어떻게 생간하세요"],
            ["오늘 날시가 정말 좋네요"],
            ["점심 메뉴는 뭐가 조을까요"]
        ],
        inputs=[input_text]
    )
    
    # 사용법 안내
    gr.Markdown(
        "### 💡 사용법\n"
        "1. **규칙 기반 + 모델**: 일반적인 오타는 규칙으로, 복잡한 경우는 모델로 처리\n"
        "2. **mT5 모델만**: 순수하게 mT5 모델만 사용 (더 느리지만 다양한 케이스 처리 가능)\n"
        "3. **더 나은 성능**: `colab_training.ipynb`로 한국어 데이터에 특화된 모델 훈련 권장"
    )
    
    # 이벤트 연결
    correct_btn.click(
        fn=gradio_demo_correction,
        inputs=[input_text, use_model],
        outputs=[output_text, processing_info]
    )

# 인터페이스 실행
print("🖥️ Gradio 데모 인터페이스를 시작합니다...")
demo.launch(share=True, debug=True)

## 📊 성능 참고사항

In [ ]:
# 성능 비교 테스트
print("⚡ 성능 비교 테스트")
print("=" * 50)

test_text = "안녕하셰요. 오늘 날시가 정말 좋네요!"

# 규칙 기반 + 모델
times_rule = []
for _ in range(5):
    start = time.time()
    result_rule = demo_correction_with_simple_rules(test_text)
    times_rule.append(time.time() - start)

# 모델만
times_model = []
for _ in range(5):
    start = time.time()
    result_model = simple_correct_text(test_text)
    times_model.append(time.time() - start)

print(f"테스트 텍스트: {test_text}")
print(f"\n📊 성능 비교:")
print(f"규칙 기반 + 모델:")
print(f"  - 결과: {result_rule}")
print(f"  - 평균 시간: {np.mean(times_rule)*1000:.1f}ms")
print(f"\nmT5 모델만:")
print(f"  - 결과: {result_model}")
print(f"  - 평균 시간: {np.mean(times_model)*1000:.1f}ms")

import numpy as np

## 🚀 다음 단계

이 데모는 기본적인 텍스트 교정 기능을 보여줍니다. 더 나은 성능을 위해서는:

### 1. 📚 전체 훈련 파이프라인
- `colab_training.ipynb` 노트북을 사용하여 한국어 데이터로 모델 파인튜닝
- LoRA를 활용한 효율적인 파라미터 튜닝
- 다양한 도메인의 한국어 데이터 활용

### 2. 🔍 전용 추론 시스템
- `colab_inference.ipynb` 노트북으로 훈련된 모델 테스트
- 성능 벤치마크 및 정량적 평가
- 배치 처리 및 최적화된 추론

### 3. 🎯 실제 적용
- 특정 도메인(뉴스, 구어체, 문어체 등)에 특화된 모델 훈련
- API 서버 구축 및 프로덕션 배포
- 지속적인 모델 개선 및 업데이트

### 📁 파일 구조
```
📁 FineTuningLLM/
├── 🧪 colab_demo.ipynb       ← 현재 데모 (기본 테스트)
├── 🚀 colab_training.ipynb   ← 모델 훈련 (T4 GPU 최적화)
└── 🔍 colab_inference.ipynb  ← 모델 추론 및 평가
```

**시작 순서**: 데모 → 훈련 → 추론 순으로 진행하시면 됩니다! 🎉